<a href="https://colab.research.google.com/github/tanogeor/Boston-Cafe-s-Trip/blob/main/Boston_Cafes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

import pandas as pd

import numpy as np

!pip install geopy

from geopy.geocoders import Nominatim

import json

import folium

import requests



In [2]:
boston = pd.read_csv('/content/Boston_Neighborhoods.csv')

In [3]:
boston.head()

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength
0,27,Roslindale,1605.568237,15,2.51,6.993827e+07,53563.912597
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937161
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724113
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757148
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493


In [4]:
CLIENT_ID = 'UNZ3EQ3HNV42BHCTULUXOLQM0XCDZLMYZOJXGKWTRH3JPDF0' # your Foursquare ID
CLIENT_SECRET = 'NMS2WNECQN10MV2YYBSSQ2QXEF5O0ZA40IZMQT4AO1P1PI1Q' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UNZ3EQ3HNV42BHCTULUXOLQM0XCDZLMYZOJXGKWTRH3JPDF0
CLIENT_SECRET:NMS2WNECQN10MV2YYBSSQ2QXEF5O0ZA40IZMQT4AO1P1PI1Q


In [5]:
address = 'Roslindale, Boston, MA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.2912093 -71.1244966


In [6]:
boston.shape

(26, 7)

Let's drop 'Harbor Islands' as their coordinates cannot be specified

In [7]:
boston.drop(labels = 25, inplace = True)

Now to get the latitude and longtitude of every Neighbourhood


In [8]:
for element in range(len(boston['Name'])):
  address = boston.loc[element, 'Name'] + ' Boston, MA, USA'

  geolocator = Nominatim(user_agent="foursquare_agent")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  boston.loc[element, 'lat'] = location.latitude
  boston.loc[element, 'long'] = location.longitude

In [9]:
boston.head()

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength,lat,long
0,27,Roslindale,1605.568237,15,2.51,6.993827e+07,53563.912597,42.291209,-71.124497
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937161,42.309820,-71.120330
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724113,42.332560,-71.103608
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757148,42.341533,-71.110155
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493,42.350011,-71.066948


In [10]:
# To visualize the map and have a graphic overview of the problem

# create map of Boston using latitude and longitude values
map_boston = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(boston['lat'], boston['long'], boston['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

In [11]:
# the function to return all the venues of a given area

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
boston_venues = getNearbyVenues(names = boston['Name'],
                                   latitudes = boston['lat'],
                                   longitudes = boston['long']
                                  )

Roslindale
Jamaica Plain
Mission Hill
Longwood
Bay Village
Leather District
Chinatown
North End
Roxbury
South End
Back Bay
East Boston
Charlestown
West End
Beacon Hill
Downtown
Fenway
Brighton
West Roxbury
Hyde Park
Mattapan
Dorchester
South Boston Waterfront
South Boston
Allston


In [24]:
print(boston_venues.shape)
boston_venues.head()

(27, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jamaica Plain,42.309820,-71.120330,Espresso Yourself,42.310199,-71.115422,Coffee Shop
1,Jamaica Plain,42.309820,-71.120330,Caffè Nero1,42.310968,-71.114786,Coffee Shop
2,Mission Hill,42.332560,-71.103608,Peet's Coffee & Tea,42.336144,-71.100423,Coffee Shop
3,Longwood,42.341533,-71.110155,Caffè Nero2,42.339201,-71.107859,Coffee Shop
4,Longwood,42.341533,-71.110155,Starbucks1,42.338709,-71.106242,Coffee Shop


Among all the different kinds of venues we have to identify the ones that correspond to 'Coffee shops'

In [14]:
pd.unique(boston_venues['Venue Category'])

array(['Scenic Lookout', 'Pizza Place', 'Cuban Restaurant', 'Pool',
       'Big Box Store', 'Rental Car Location', 'Donut Shop', 'Trail',
       'Theater', 'American Restaurant', 'Accessories Store',
       'Coffee Shop', 'Bookstore', 'Bakery', 'Liquor Store', 'Pet Store',
       'Deli / Bodega', 'Seafood Restaurant', 'Art Gallery',
       'Noodle House', 'Library', 'Bar', 'Child Care Service',
       'Home Service', 'Park', 'Italian Restaurant', 'Ice Cream Shop',
       'New American Restaurant', 'Sandwich Place', 'Greek Restaurant',
       'Gastropub', 'Pub', 'Sushi Restaurant', 'Caribbean Restaurant',
       'Café', 'Grocery Store', 'Vegetarian / Vegan Restaurant',
       'Fast Food Restaurant', 'Yoga Studio', 'Falafel Restaurant',
       'Bagel Shop', 'Hostel', 'Diner', 'Hotel', 'Shipping Store',
       'Pharmacy', 'Bank', 'Bubble Tea Shop', 'Food Court', 'Spa',
       'Performing Arts Venue', 'Comedy Club', 'Lounge',
       'French Restaurant', 'Hotel Bar', 'Gourmet Shop', 'Wine S

Let's drop every venue different to 'Coffee Shop'

In [15]:
boston_venues.drop(boston_venues[boston_venues['Venue Category'] != 'Coffee Shop'].index, inplace = True) 

# reset the index
boston_venues.reset_index(drop = True, inplace = True)
boston_venues.iloc[:,0].head()

0    Jamaica Plain
1    Jamaica Plain
2     Mission Hill
3         Longwood
4         Longwood
Name: Neighborhood, dtype: object

In [16]:
# this is done to rename the duplicate values of Venues column
# if this is skipped an error will arise during Route Optimization

from collections import Counter # Counter counts the number of occurrences of each item
from itertools import tee, count

def uniquify(seq, suffs = count(1)):
    """Make all the items unique by adding a suffix (1, 2, etc).

    `seq` is mutable sequence of strings.
    `suffs` is an optional alternative suffix iterable.
    """
    not_unique = [k for k,v in Counter(seq).items() if v>1] # so we have: ['name', 'zip']
    # suffix generator dict - e.g., {'name': <my_gen>, 'zip': <my_gen>}
    suff_gens = dict(zip(not_unique, tee(suffs, len(not_unique))))  
    for idx,s in enumerate(seq):
        try:
            suffix = str(next(suff_gens[s]))
        except KeyError:
            # s was unique
            continue
        else:
            seq[idx] += suffix

uniquify(boston_venues['Venue'])
# assert boston_venues['Venue'] == check  # raise an error if we failed
boston_venues['Venue']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0                   Espresso Yourself
1                         Caffè Nero1
2                 Peet's Coffee & Tea
3                         Caffè Nero2
4                          Starbucks1
5                          Starbucks2
6     Jaho Coffee Roaster & Wine Bar1
7                    Gracenote Coffee
8               The Well Coffee House
9                         Caffè Nero3
10                        Caffè Nero4
11    Jaho Coffee Roaster & Wine Bar2
12               George Howell Coffee
13                       Thinking Cup
14                      Render Coffee
15                         Starbucks3
16                        Blue Bottle
17                         Starbucks4
18                             Zume's
19                        Cuppacoffee
20                         Starbucks5
21                         Cafenation
22                         Starbucks6
23               P.S. Gourmet Coffee1
24                        Caffè Nero5
25               P.S. Gourmet Coffee2
26          

In [17]:
print(boston_venues.shape)
boston_venues

(27, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jamaica Plain,42.309820,-71.120330,Espresso Yourself,42.310199,-71.115422,Coffee Shop
1,Jamaica Plain,42.309820,-71.120330,Caffè Nero1,42.310968,-71.114786,Coffee Shop
2,Mission Hill,42.332560,-71.103608,Peet's Coffee & Tea,42.336144,-71.100423,Coffee Shop
3,Longwood,42.341533,-71.110155,Caffè Nero2,42.339201,-71.107859,Coffee Shop
4,Longwood,42.341533,-71.110155,Starbucks1,42.338709,-71.106242,Coffee Shop
5,Longwood,42.341533,-71.110155,Starbucks2,42.338562,-71.106230,Coffee Shop
6,Bay Village,42.350011,-71.066948,Jaho Coffee Roaster & Wine Bar1,42.351720,-71.062820,Coffee Shop
7,Leather District,42.352322,-71.057343,Gracenote Coffee,42.351657,-71.058204,Coffee Shop
8,Leather District,42.352322,-71.057343,The Well Coffee House,42.351515,-71.055021,Coffee Shop
9,Leather District,42.352322,-71.057343,Caffè Nero3,42.353521,-71.062348,Coffee Shop


Let's visualize the venues on a map

In [25]:
# add markers to map
map1 = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, label in zip(boston_venues['Venue Latitude'], boston_venues['Venue Longitude'], boston_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1) 
      
map1

26 coffee shops seems like a small number to accommodate such a big metropolis as Boston.
I could have a bigger radius when getting the venues
But let's work with 26, as it seems as a reasonable (if not too big) number for a salesman to pass through

The following pieces of code are borrowed from Gurobi's official site

You'll find their take on the TSP here: https://www.gurobi.com/resource/traveling-salesman-problem/

In [19]:
# To change the above dataframe in a set of lists and dictionaries that can later be read by the corresponding functions

venues = []
coordinates = {}

for i in range(boston_venues.shape[0]):
  venue = boston_venues.loc[i, 'Venue']
  venues.append(venue)
  coordinates[venue] = (float(boston_venues.loc[i, 'Venue Latitude']), float(boston_venues.loc[i, 'Venue Longitude']))

In [20]:
# Preprocessing
# The following function calculates the distance of each pair of venues combination.

import math
from itertools import combinations,product

# Compute pairwise distance matrix

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in product(venues, venues) if c1 != c2}

# Callback Definition
# The following function determines lazy constraints to eliminate sub-tours.
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < len(venues):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)


# Finding Shortest Route
# The following function determines the shortest route from a given set of edges.
# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = venues[:]
    cycle = venues[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

Model Deployment
We now determine the model for the TSP, by defining decision variables, constraints, and objective function. 
Next, we start the optimization and Gurobi finds the optimal route for the TSP.

In [21]:
!pip install -i https://pypi.gurobi.com gurobipy 
from gurobipy import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7 & Gurobi 9.1.1

m = gp.Model()

# Variables: is city 'i' adjacent to city 'j' on the tour?

vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='e')
for i, j in vars.keys():
    m.addConstr(vars[j, i] == vars[i, j])  # edge in opposite direction

# Constraints: two edges incident to each city

m.addConstrs(vars.sum(c, '*') == 2 for c in venues)

# Optimize the model

m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)



Looking in indexes: https://pypi.gurobi.com
Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 729 rows, 702 columns and 2106 nonzeros
Model fingerprint: 0x33a3c4b9
Variable types: 0 continuous, 702 integer (702 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-04, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 702 rows and 351 columns
Presolve time: 0.00s
Presolved: 27 rows, 351 columns, 702 nonzeros
Variable types: 0 continuous, 351 integer (351 binary)

Root relaxation: objective 4.440790e-01, 33 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

We retrieve the optimal solution of the TSP and verify that the optimal route (or tour) goes to all the cities and returns to the origin city.


In [22]:
# Retrieve solution

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(venues)


The optimal route is displayed in the following map.

In [23]:
# To visualize the venues
# create map of Boston using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(boston_venues['Venue Latitude'], boston_venues['Venue Longitude'], boston_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[1])

folium.PolyLine(points).add_to(map)

map

